In [153]:
import numpy as np

data = open('./data/pg.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} uniques')

ctoi = {ch:i for i, ch in enumerate(chars)}
itoc = {i:ch for i, ch in enumerate(chars)}

data has 50015 characters, 76 uniques


In [154]:
# hyperparameters
hidden_size = 100
seq_len = 25
learning_rate = 0.2

# model params
w_xh = np.random.randn(hidden_size, vocab_size)*0.01 # input -> hidden
w_hh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden -> hidden
w_hy = np.random.randn(vocab_size, hidden_size)*0.01 # hidden -> output
b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((vocab_size, 1)) # output bias


In [155]:
def calc_loss(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  
  # forward pass
  for t in range(len(inputs)):
    # one-hot encoding
    xs[t] = np.zeros((vocab_size, 1))
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(w_xh, xs[t]) + np.dot(w_hh, hs[t-1]) + b_h) # hidden state
    ys[t] = np.dot(w_hy, hs[t]) + b_y # log probs for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probs for next chars
    loss += -np.log(ps[t][targets[t], 0]) # softmax

  # backward pass
  dw_xh, dw_hh, dw_hy = np.zeros_like(w_xh), np.zeros_like(w_hh), np.zeros_like(w_hy)
  db_h, db_y = np.zeros_like(b_h), np.zeros_like(b_y)
  d_hnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    # https://cs231n.github.io/neural-networks-case-study/#grad
    dy[targets[t]] -= 1 # backprop into Y
    dw_hy += np.dot(dy, hs[t].T)
    db_y += dy
    dh = np.dot(w_hy.T, dy) + d_hnext # backprop into h
    d_hraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh non-linearity
    db_h += d_hraw
    dw_xh += np.dot(d_hraw, xs[t].T)
    dw_hh += np.dot(d_hraw, hs[t-1].T)
    d_hnext = np.dot(w_hh.T, d_hraw)

  for dparam in [dw_xh, dw_hh, dw_hy, db_h, db_y]:
    np.clip(dparam, -5, 5, out=dparam) # prevent exploding gradients
  return loss, dw_xh, dw_hh, dw_hy, db_h, db_y, hs[len(inputs)-1]


def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  indices = []
  for t in range(n):
    h = np.tanh(np.dot(w_xh, x) + np.dot(w_hh, h) + b_h)
    y = np.dot(w_hy, h) + b_y
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    indices.append(ix)
  return indices

In [156]:
n, p = 0, 0
mw_xh, mw_hh, mw_hy = np.zeros_like(w_xh), np.zeros_like(w_hh), np.zeros_like(w_hy)
mb_h, mb_y = np.zeros_like(b_h), np.zeros_like(b_y)
smooth_loss = -np.log(1.0/vocab_size)*seq_len

In [ ]:
n, p = 0, 0
iters = 50000

while n < iters:
  # prepare inputs to go from left-to-right in steps (seq_len)
  if p+seq_len+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size, 1)) # reset RNN memory
    p = 0 # start at beginning of data
  inputs = [ctoi[ch] for ch in data[p:p+seq_len]]
  targets = [ctoi[ch] for ch in data[p+1:p+seq_len+1]]

  if n % 1000 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(itoc[ix] for ix in sample_ix)
    print(f'----\n {txt} \n----')

  # forward seq_len characters through the net and fetch gradient
  loss, dw_xh, dw_hh, dw_hy, db_h, db_y, hprev = calc_loss(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001 # TODO: why is this good?

  if n % 1000 == 0:
    print(f'iter {n}, loss: {smooth_loss}')

  # Adagrad parameter update
  for param, dparam, mem in zip([w_xh, w_hh, w_hy, b_h, b_y],
                                [dw_xh, dw_hh, dw_hy, db_h, db_y],
                                [mw_xh, mw_hh, mw_hy, mb_h, mb_y]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
  p += seq_len
  n += 1


----
 a mo bere lostan thegfn'p bhiing startu tulipt ere fat anghacf seocore ondel't tave do it ptere to. bo o ware larke vepy meebs iser Ininzuse; thento thalrps ut sov there'lyn find what at os bis suess  
----
iter 0, loss: 54.84139602616046
